# Simple TF model creation using resource variables 

In [16]:
import numpy as np
import tensorflow as tf
import os

In [8]:
class CompareAndAccumulate(tf.Module):
  """
      Accumulates a given value to the resource variable array (initialized as 0.).
      Accumulates add/subtract based on second boolean input.
  """

  def __init__(self, name):
    super().__init__(name=name)
    self._accum = tf.Variable(
        initial_value=np.zeros((100,), dtype=np.float32),
        trainable=False,
        name="Accumulator",
        dtype=tf.float32,
        shape=[100],
    )

  @tf.function(
      input_signature=[
          tf.TensorSpec(shape=[100], dtype=tf.float32, name="accum_val"),
          tf.TensorSpec(shape=[1], dtype=tf.bool, name="accumulate_add"),
      ]
  )
  def __call__(self, accum_val, accumulate_add):
    if accumulate_add:
      self._accum.assign_add(accum_val)
    else:
      self._accum.assign_sub(accum_val)
    return self._accum.read_value()


In [9]:
class CompareAndAccumulateKerasLayer(tf.keras.layers.Layer):
  """Accumulates a given value to the resource variable array (initialized as 0.).

  Accumulates add/subtract based on second boolean input.
  """

  def __init__(self, name):
    super().__init__(name=name)
    self._accum = tf.Variable(
        initial_value=[np.zeros((100,), dtype=np.float32)],
        trainable=False,
        name="Accumulator",
        dtype=tf.float32,
        shape=(1, 100),
    )

  def call(self, accum_val, accumulate_add):
    @tf.function
    def conditional_accumulate(accum_val, accumulate_add):
      if accumulate_add:
        self._accum.assign_add(accum_val)
      else:
        self._accum.assign_sub(accum_val)
    conditional_accumulate(accum_val, accumulate_add)
    return self._accum.read_value()


In [10]:
model = CompareAndAccumulate("CompareAndAccumulate")

2023-10-13 15:16:30.900229: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3372 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 950M, pci bus id: 0000:01:00.0, compute capability: 5.0


In [11]:
concrete_func = model.__call__.get_concrete_function()
print(concrete_func)

ConcreteFunction __call__(accum_val, accumulate_add)
  Args:
    accum_val: float32 Tensor, shape=(100,)
    accumulate_add: bool Tensor, shape=(1,)
  Returns:
    float32 Tensor, shape=(100,)


In [14]:
def get_model_from_concrete_function():
  """Accumulator model built via TF concrete functions."""
  model = CompareAndAccumulate("CompareAndAccumulate")
  concrete_func = model.__call__.get_concrete_function()
  converter = tf.lite.TFLiteConverter.from_concrete_functions(
      [concrete_func], model
  )
  return converter.convert()

In [15]:
def get_model_from_keras():
  """Accumulator model built via Keras custom layer."""
  input_layer_int = tf.keras.layers.Input(
      shape=[100], dtype=tf.float32, name="accum_val"
  )
  input_layer_bool = tf.keras.layers.Input(
      shape=[1], dtype=tf.bool, name="accumulate_add"
  )
  accumulate_out = CompareAndAccumulateKerasLayer("CompareAndAccumulate")(
      input_layer_int, input_layer_bool
  )

  model = tf.keras.models.Model(
      inputs=[input_layer_int, input_layer_bool], outputs=accumulate_out
  )
  converter = tf.lite.TFLiteConverter.from_keras_model(model)
  return converter.convert()


Save the tflite model

In [19]:
if not os.path.exists("models"):
    os.makedirs("models")
save_path = "models/accu.tflite"
with open(save_path, "wb") as f:
    f.write(tflite_model)
print("Tflite model saved to models/accu.tflite")

Tflite model saved to models/accu.tflite
